## LDA(Latent Dirichlet Allocation), 잠재디리클레할당
    - 토픽 모델링에 사용되는 대표적인 알고리즘
    
- 모듈
    - gensim.models.wrappers.LdaMallet 모듈사용 => 깁스 샘플링을 사용하여 훈련 코퍼스에서 LDA 모델 추천가능
        - 깁스 샘플링 => 2번째로 생성될 표본은 1차 생성 표본의 영향을 받으며, 나머지 변수는 가만히 두고 한 변수에만 변화를 준다.
            - ex) p(x1, y1, z1)가 존재할 경우, 변수명+표본의 차수
            - 변경 순서 => p(y1, z1)을 유지시킨 채 x1 -> x2 로 변경 => p(x2, z1)을 유지시킨 채 y1 -> y2 로 변경 => ...
            - 1차 변경 결과 => p(x2, y2, z2)

        - 검색창에 mallet.cs.umass.edu/dist/mallet-2.0.8.zip 치면 해당 zip파일 자동다운로드
        
        
- CoherenceMetric
	- 토픽 내 일관성 지수를 계산, 최적의 토픽 개수를 찾기위해서 사용
	- coherence = 'c_v' 사용
	- score가 높은 토픽 개수 사용
    

- Corpora 라이브러리
	- from gensim import corpora / from gensim.corpora.dictionary import Dictionary
	- 텍스트 데이터의 특성 상 수치화해서 분석 및 연산 진행해야함.
	- 텍스트를 가지고 분석이나 연산에 용이하게 데이터 집합을 만들어 주는 라이브러리
	- document = ['a', 'b', 'c']라는 리스트가 존재할 때
	- dict = corpora.Dictionary(document) / dict = Dictionary(document) 
	- dict는 리스트가 사전형식으로 변환 (ex) {'a': 0, 'b': 1, 'c': 2}
	- doc2bow를 통해서 단어를 숫자로 변경하고 카운트까지 알 수 있음
	- dict.doc2bow(input) 으로 input에 있는 텍스트 사전을 통해 단어별 개수 확인
	- ex) [(0, 1), (1, 5)]이면 0번에 위치한 단어는 1번, 1번에 위치한 단어는 5번 나왔다는 의미

In [1]:
# !pip install konlpy
# 3.8.5 버전이후로 gensim.models.wrappers.LdaMallet 삭제 => 3.8.3 버전 사용
# !pip install gensim==3.8.3

In [1]:
from tqdm.notebook import tqdm # 진행률 프로세스바
import itertools # 효율적인 루핑을 위한 이터레이터
from gensim.models.ldamodel import LdaModel # LDA 모델
from gensim.models.callbacks import CoherenceMetric 
import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel # 최적의 토픽개수 구하는 모델
import pandas as pd
import numpy as np
from konlpy.tag import Okt
import os

In [2]:
# 전체 데이터 읽기
# df = pd.read_parquet('after_preprocessing.parquet')

# 필요한 컬럼만 읽고 싶을 경우
df = pd.read_parquet('after_preprocessing.parquet', columns=['source', 'content'])

In [3]:
# Okt로 토큰화
def tokenizedOkt(tourDf):
    okt = Okt()
    tour_tokenized = [] 

    # 해당 관광지에 관한 content들 토큰화
    tqdmDf = tqdm(tourDf)

    for tourInfo in tqdmDf:
        tqdmDf.set_description(f'Processing tokenized')
        tokenized_doc = okt.pos(tourInfo)
        
        # 명사만 사용
        tokenized_list = [w for w,t in tokenized_doc if t =='Noun']
        tour_tokenized.append(tokenized_list)
        
    return tour_tokenized

In [4]:
# 토크나이징한 데이터를 입력으로 받아서, 이 데이터들을 바탕으로 사전 제작 
# corpus에는 (token_id, token_count)가 튜플형태로 저장
def dictionary(data_word):
    
    # 토큰화된 문장리스트로 사전구성
    id2word=corpora.Dictionary(data_word)
    id2word.filter_extremes(no_below = 20) # 20회 이하로 등장한 단어는 삭제
    texts = data_word
    # 토큰화된 문장리스트의 각 단어를 통해서 해당 단어가 총 몇번 등장하는 지 계산
    corpus=[id2word.doc2bow(text) for text in texts]
    return id2word, texts, corpus

In [12]:
# 최적의 토픽 개수 도출
def compute_coherence_values(dictionary, corpus, texts, start, limit, step):
    
    os.environ.update({'MALLET_HOME':r'C:/pythonPrj/jejuModel/mallet-2.0.8/'})
    
    # 설치한 mallet 경로
    mallet_path = 'C:/pythonPrj/jejuModel/mallet-2.0.8/bin/mallet'
    
    # 토픽 개수에 따른 모델 저장
    model_list = []
    
    # 모델별 일관성 지수 저장
    coherence_values = []
    
    print(f'start: {start}, limit: {limit}, step: {step}')

    # 진행상황 확인 변수 선언
    tqdmSet = tqdm(range(start, limit, step))
    
    for num_topics in tqdmSet:
        # 진행 상황 확인(현재 진행되는 num_topics로 해당 문자열 변경)
        tqdmSet.set_description(f'Processing num_topics')
        
        # 토픽 개수별 모델 
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=dictionary)
        model_list.append(model)
        
        # 모델별 토픽의 일관성 지수 계산 모델
        coherencemodel = CoherenceModel(model=model, texts=data_word, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
        
    return model_list, coherence_values

In [13]:
def LDA(data_word, Data_list, start, limit, step):
    id2word, texts, corpus = dictionary(data_word)
    
    print("Create LDA instance")
    os.environ.update({'MALLET_HOME':r'C:/pythonPrj/jejuModel/mallet-2.0.8/'})
    
    # 설치한 mallet 경로
    mallet_path = 'C:/pythonPrj/jejuModel/mallet-2.0.8/bin/mallet'
    
#     # LDA 모델
#     ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)
    
#     # 토픽의 일관성 지수 계산 
#     coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=texts, dictionary=id2word, coherence='c_v')
#     coherence_ldamallet = coherence_model_ldamallet.get_coherence()
    
    # 해당 반복문 사용 불가할 시 위의 코드 사용
    # 위의 compute_coherence_values를 통한 최적의 토픽 개수 사용
    model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=texts, start=start, limit=limit, step=step)
    
    x = range(start, limit, step)
    topic_num = 0
    count = 0
    max_coherence = 0
    for m, cv in zip(x, coherence_values):
        # score가 높을수록 좋음
        print("Num Topics =", m, " has Coherence Value of", cv)
        coherence = cv
        if coherence >= max_coherence:
            max_coherence = coherence
            topic_num = m
            model_list_num = count   
        count = count+1
        
    # 최적의 모델 도출
    optimal_model = model_list[model_list_num]
    model_topics = optimal_model.show_topics(formatted=False, num_words=10)
    return optimal_model, model_topics

### 100개 관광지에 대한 토픽 모델링 진행
    - 추천 대상 100개의 관광지 선정 (2021년 이후 블로그 글 수 순위, 계절별 특성비율, 타 사이트기준 제주도 관광지 순위 등 고려)

In [6]:
tourlist = ['아르떼뮤지엄',
 '산방산',
 '스누피가든',
 '함덕해수욕장',
 '사계해변',
 '섭지코지',
 '도두동 무지개 해안도로',
 '협재해수욕장',
 '동백포레스트',
 '9.81 파크',
 '원동',
 '엉덩물계곡',
 '새별오름',
 '구좌해안로',
 '도치돌 알파카목장',
 '한라산 아래 첫 마을',
 '곶자왈',
 '쇠소깍',
 '송악산',
 '윈드1947 카트 테마파크',
 '뽀로로&타요 테마파크 제주',
 '탑동',
 '이호테우해수욕장',
 '판포포구',
 '용머리해안',
 '백록담',
 '신화테마파크',
 '도두봉',
 '삼양해수욕장',
 '서귀다원',
 '카멜리아힐',
 '안돌오름',
 '광치기해변',
 '비자림',
 '사려니숲길',
 '사진놀이터',
 '관음사',
 '비양도',
 '김녕 청굴물',
 '신창풍차해안도로',
 '가파도',
 '수월이못',
 '해맞이해안로',
 '제주센트럴파크',
 '서귀포해안',
 '중문색달해수욕장',
 '금악오름(왕매)',
 '쁘램 요가',
 '함덕카페거리',
 '브릭캠퍼스',
 '곽지해수욕장',
 '용두암',
 '제주 카카오캠핑',
 '닭머르해안길',
 '금능해수욕장',
 '노을해안로',
 '아침미소목장',
 '서우봉',
 '휴림',
 '도두해안도로',
 '제주토종흑염소목장',
 '취다선리조트',
 '휴애리 자연생활공원',
 '액티브파크',
 '천지연폭포',
 '화조원',
 '민속해안로',
 '김택화미술관',
 '석부작박물관',
 '오조포구',
 '바이나흐튼 크리스마스 박물관',
 '이호테우말등대',
 '평대해변',
 '법환포구',
 '이상한 나라의 앨리스',
 '제주카약',
 '제주도립김창열미술관',
 '혼인지',
 '정방폭포',
 '한라산영실코스',
 '포토갤러리 자연사랑미술관',
 '오설록티뮤지엄',
 '막숙',
 '소천지',
 '신양섭지해수욕장',
 '목장카페 드르쿰다',
 '이로이로공방',
 '수월봉',
 '신화워터파크',
 '제주제트',
 '애월한담해안산책로',
 '김녕해수욕장',
 '수산저수지',
 '주상절리대(중문대포해안)',
 '톨칸이',
 '에코랜드 테마파크',
 '성산일출봉(UNESCO 세계자연유산)',
 '천아숲길 천아계곡',
 '종달리수국길' ,
 '녹산로유채꽃길'] 

In [7]:
len(tourlist)

100

In [8]:
tourDf = df[df['source'].isin(tourlist)]

In [9]:
len(tourDf['source'].unique())

100

In [10]:
# seires => DataFrame으로 변환 (원본 데이터)
# tour_OriDf = pd.DataFrame(tourDf)

# 토큰화 진행
tour_tokenized = tokenizedOkt(tourDf['content'])

data_word = tour_tokenized   # 토큰화 데이터
Data_list = tourDf             # 원본 데이터

  0%|          | 0/78931 [00:00<?, ?it/s]

In [14]:
start=100; limit=101; step=1;  
model, topics = LDA(data_word, Data_list, start, limit, step)

Create LDA instance
start: 100, limit: 101, step: 1


  0%|          | 0/1 [00:00<?, ?it/s]

Num Topics = 100  has Coherence Value of 0.6121278170310995


In [15]:
# 결과확인 (100개 관광지)
model.show_topics(formatted=False, num_topics=100)

[(0,
  [('아이', 0.19469771226720423),
   ('엄마', 0.09470880878844483),
   ('아빠', 0.05362856244567328),
   ('가족', 0.05109485676240499),
   ('아들', 0.04338277450019419),
   ('우리', 0.023145493887666218),
   ('부모님', 0.014217417839507314),
   ('둘째', 0.012534445451351002),
   ('어른', 0.00976956224223706),
   ('데리', 0.009247101033825896)]),
 (1,
  [('함덕', 0.18810375402001744),
   ('해수욕장', 0.08477052087403444),
   ('봉', 0.07578972678909561),
   ('서우', 0.06297977217396532),
   ('바다', 0.055375552282768775),
   ('조천읍', 0.05452195605782814),
   ('제주시', 0.04341318265155841),
   ('닭머르', 0.030200474888040636),
   ('해변', 0.02956929458086622),
   ('델문', 0.02027591596285053)]),
 (2,
  [('친구', 0.14662947307172317),
   ('달', 0.033377258284416914),
   ('살기', 0.028283104866542307),
   ('살이', 0.02326881499609239),
   ('오늘', 0.019543750962640973),
   ('혼자', 0.019001819747973464),
   ('마음', 0.014905960672907432),
   ('기억', 0.009783284559524014),
   ('지금', 0.009212830649347687),
   ('추억', 0.009138671641024764)]),
 

#### 결과: 100개의 토픽 중 일부는 한 관광지의 토픽으로 볼 수 있지만 대부분은 한 관광지에 대한 토픽으로 보기 힘들다.